# Object Detection with RHODS

## Demo Setup

Prerequisite: You have deployed the [RHODS Demo Pack](https://github.com/mamurak/os-mlops/tree/main#rhods-demo-pack) and cloned this repository within a RHODS workbench using the `object detection` workbench image in the `object detection` Project. GPU enablement is optional.

### Bucket setup

Before proceeding with the demo, ensure the `object-detection` bucket exists in your S3 storage, e.g. in your Minio instance that is referenced in the `object-detection` data connection. In case of Minio:
- Call the Minio UI route URL (OCP console: `Networking` -> `Routes` -> `Project`: `minio` -> `minio-ui`).
- Log in (use `minio` and `minio123` in case of the RHODS Demo Pack).
- Create the `object-detection` bucket in the `Buckets` tab.

### GPU-accelerated inferencing

If you have enabled GPUs in your cluster, configure GPU usage in your Triton model server instance:
- In the RHODS dashboard, navigate to project `object detection` and enter the context menu of the `Triton` model server instance (three dots on the right).
- Select _Edit model server_.
- Increase the count of `Model server GPUs`.

### Offline scoring data

To upload sample images for the offline scoring pipeline, run the following cell.

In [ ]:
from glob import glob
from os import environ, path
from tempfile import TemporaryDirectory

from boto3 import client
from openimages.download import download_dataset


# Image count per class. Overall sample size will be limit x number of classes.
limit = 10

s3_endpoint_url = environ.get('AWS_S3_ENDPOINT')
s3_access_key = environ.get('AWS_ACCESS_KEY_ID')
s3_secret_key = environ.get('AWS_SECRET_ACCESS_KEY')
s3_bucket_name = environ.get('AWS_S3_BUCKET')

s3_client = client(
    's3', aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    endpoint_url=s3_endpoint_url,
)

with TemporaryDirectory() as images_folder:
    download_dataset(
        images_folder,
        class_labels=['Bicycle', 'Car', 'Traffic sign'],
        annotation_format='darknet',
        limit=limit
    )

    image_file_paths = glob(path.join(images_folder, '**/*.jpg'), recursive=True)
    print(f'Uploading images: {image_file_paths}')
    image_names = [
        file_path.split('/')[-1]
        for file_path in image_file_paths
    ]
    for index, image_name in enumerate(image_names):
        image_file_path = image_file_paths[index]
        s3_client.upload_file(image_file_path, s3_bucket_name, image_name)
        print(f'Uploaded {image_name}')

You can now proceed with the demo and follow the end-to-end demo flow.

## Model training

Show how to create and run training pipelines with Elyra and Data Science Pipelines:
1. In the `model-training` folder, open the appropriate Elyra pipeline, i.e. `model-training-gpu.pipeline` if you have enabled GPUs in your cluster, else `model-training-cpu.pipeline`.
2. Submit the pipeline to the Data Science Pipelines backend (via `Run Pipeline`).
3. Track the pipeline progress through the list of triggered runs in the RHODS dashboard.
4. Once the pipeline run has completed, you can find the trained model in the `object-detection` bucket.

## Online scoring

Deploy the model through RHODS Model Serving:
1. In the RHODS dashboard in project `object-detection`, select `Deploy model` in the appropriate model server, i.e. `Triton` if you enabled GPUs in your cluster, else `OVMS`.
2. Enter `demo-model` under _Model Name_, `ONNX` as _Model framework_, existing data connection `object-detection`, and `model-latest.onnx` as _Folder path_.
3. Select _Deploy_.
4. Copy the _Inference endpoint_ URL.
5. Open the `online-scoring` notebook.
6. Paste the inference endpoint URL into the placeholder of the `prediction_url` variable.
7. Run the notebook.

## Offline scoring

Show how to deploy scoring pipelines through Elyra and Data Science Pipelines:
1. Open the `offline-scoring` notebook.
2. Submit the pipeline to the Data Science Pipelines backend (via `Run Pipeline`).
3. Track the pipeline progress through the list of triggered runs in the RHODS dashboard.
4. Once the pipeline run has completed, you can find the classification results in the `object-detection` bucket as `prediction-[timestamp].csv`.